To install a package over pip so that it works in the notebook you have to run: 
```bash
$ source /opt/tljh/user/bin/activate
$ pip install <package name>
```
and then restart the ipykernal.

Imports

In [1]:
import ferryFile
import port
import numpy as np
import heapq

Constants and Objects

In [2]:
MAX_SIMULATION_TIME = 1000


PORT_CODES = ["SWB", "TSA", "SGI", "HSB", "NAN", "LNG"]
FERRIES = []
SEEDS = [234,4234,324,325543,2]

# Programmatically create the port objects for each of the port codes
for port_code in PORT_CODES:
    globals()[port_code] = port.Port() 
    globals()[port_code].port_code = port_code


# Most of the trips take the same amount of time independent of the ferry 
# Makes every port to every other port take 20ish minutes. 
# Times are made to be the same independent of direction.
TRIP_TIMES = {}
for port in PORT_CODES:
    for other_port in [i for i in PORT_CODES if i != port]:
        if other_port + port in TRIP_TIMES.keys():
            TRIP_TIMES[port+other_port] = TRIP_TIMES[other_port+port]
        else:
            TRIP_TIMES[port + other_port] = np.random.normal(20, 2.5)

# Set the loading and unloading times
# Port + Current_time for every 5 step increment
LOADING_AND_UNLOADING_TIMES = {}
for port_code in PORT_CODES:
    for time in range(0, MAX_SIMULATION_TIME, 5):
        LOADING_AND_UNLOADING_TIMES[globals()[port_code] + time] = np.random.normal(10, 2.5)
    


In [3]:
def normal_min(value, min):
    gen_val = int(np.random.normal(value))
    while gen_val < min:
        gen_val = int(np.random.normal(value))
    return gen_val


for seed_value in SEEDS:
    np.random.seed(seed_value)

    #Create a ferry object
    QNW = ferryFile.Ferry()
    QNW.ferry_name = "Queen of New Westminster"
    QNW.ferry_code = "QNW"
    QNW.ferry_capacity = 150
    QNW.ferry_route = [TSA, SWB, SGI, TSA]
    QNW.ferry_trip_time = TRIP_TIMES
    QNW.set_expected_departure_times([0, 25, 70, 90])
    QNW.set_expected_arrival_times([0, 15, 45, 80])
    QNW.loading_unloading_time = LOADING_AND_UNLOADING_TIMES
    QNW.trips_required = 1
    
    QNW.trips_completed = 0
    QNW.ferry_current_port_index = 0
    QNW.validate_arrival_and_departure_times()


    current_time =0
    event_queue = []
    heapq.heapify(event_queue)
    heapq.heappush(event_queue, (current_time, QNW))
    

    current_event = heapq.heappop(event_queue)
    while current_event[1].next_function != None:
        current_time = current_event[0]

        next_event_time = current_event[1].next_function(current_time)
        heapq.heappush(event_queue, (next_event_time, current_event[1]))

        current_event = heapq.heappop(event_queue)

    print(f"Arrival times [scheduled, actual]: {QNW.ferry_arrival_time}")
    print(f"Departure times [scheduled, actual]: {QNW.ferry_departure_time}")








QNW has completed their trip at time 90.04966197907234.
Total time with the ferry being late to arrive: 20.541582955057475
Total time with the ferry being late to depart: 5.016366112360757
Arrival times [scheduled, actual]: [[0, 0], [15, 19.726213240464386], [45, 50.76570773552075], [80, 90.04966197907234]]
Departure times [scheduled, actual]: [[0, 0], [25, 30.016366112360757], [70, 70], [90, 0]]
QNW has completed their trip at time 90.04966197907234.
Total time with the ferry being late to arrive: 20.541582955057475
Total time with the ferry being late to depart: 5.016366112360757
Arrival times [scheduled, actual]: [[0, 0], [15, 19.726213240464386], [45, 50.76570773552075], [80, 90.04966197907234]]
Departure times [scheduled, actual]: [[0, 0], [25, 30.016366112360757], [70, 70], [90, 0]]
QNW has completed their trip at time 90.04966197907234.
Total time with the ferry being late to arrive: 20.541582955057475
Total time with the ferry being late to depart: 5.016366112360757
Arrival tim